In [1]:
import numpy as np
import scipy.linalg
import cvxpy as cp
from numpy.linalg import matrix_power

A Robust Learning Approach for Regression Models Based on Distributionally Robust Optimization

In [2]:
def dlqr(A,B,Q,R):
    """
    Calculates the optimal gain matrix K for system
    x[k+1] = A x[k] + B u[k]
    with cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k], such that
    u[k] = -K x[k]
    
    Returns: 
        -K
    """
    # first, solve the ricatti equation
    P = np.matrix(scipy.linalg.solve_discrete_are(A, B, Q, R))
    # compute the LQR gain
    K = np.matrix(scipy.linalg.inv(B.T*P*B+R)*(B.T*P*A))
    return -K

In [3]:
def disc_linear_system(A,B,delta_t):
    '''
    Discrete a linear system with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]
    
    Returns:
        Ak
        Bk
    
    ''' 
    Nx = np.shape(A)[0]
    Ix = np.identity(Nx)
    
    Ak = np.linalg.inv(Ix - delta_t * A)
    Bk = np.linalg.inv(Ix - delta_t * A) @ (delta_t * B)
    
    
    return Ak, Bk

In [4]:
def RK4_np(f, x, u, t, h):
    """
    Runge-Kutta 4th order solver using numpy array data type.

    Args:
        f: A function returning first order ODE in 2D numpy array (Nx x 1).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
    Returns:
        x_next: Vector of next value in 2D numpy array (Nx x 1)
    """
    x = np.reshape(x, (np.shape(x)[0], -1))    # Reshape x to col vector in np 2D array
    k1 = f(t, x, u)
    k2 = f(t + h / 2, x + h / 2 * k1, u)
    k3 = f(t + h / 2, x + h / 2 * k2, u)
    k4 = f(t + h, x + h * k3, u)
    x_next = x + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

In [5]:
def mass_string_ode(t, x, u):
    m = 2 #[kg]
    k1 = 3 # [N/m]
    k2 = 2 # [N/m]
    
    A = np.array([[0,1],[-k2/m, -k1/m]])
    B = np.array([[0],[1/m]])
    
    dot_x = A @ x + B @ u
    
    return dot_x

In [6]:
def stack_system(A, B, C, D, E, x_init, N):
    '''
    Stack system matrix for N prediction horizon
    
    x_next = A x_k + B u_k + C w_k
    y_k = D x_k + E w_k
    
    '''

    n = np.shape(A)[1]    #  Dimension of state
    m = np.shape(B)[1]    #  Dimension of input
    d = np.shape(C)[1]    #  Dimension of disturbance
    r = np.shape(D)[0]    #  Dimension of output
    
#     print(Nx,Nu,Nw,Ny)
    Nx = (N+1) * n
    Nu = N * m
    Ny = N * r
    Nw = N * d
    
    Ax = np.zeros([Nx,n])
    Bx = np.zeros([Nx,Nu])
    Cx = np.zeros([Nx,Nw])
    Ay = np.zeros([Ny,n])
    By = np.zeros([Ny,Nu])
    Cy = np.zeros([Ny,Nw])
    Ey = np.zeros([Ny,Nw])
    Cx_tilde = np.zeros([Nx, Nw + 1])
    Cy_tilde = np.zeros([Ny + 1, Nw + 1])
    Ey_tilde = np.zeros([Ny + 1, Nw + 1])
    D_tilde = np.zeros([Ny + 1, Nx])
#     H

# Ax
    for i in range(N+1):
        Ax[i*n:(i+1)*n,:] = matrix_power(A,i)
# Bx
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            Bx[(i + 1) * n : (i + 2) * n, (i-j) * m: (i-j+1) * m ] = mat_temp
            mat_temp = A @ mat_temp
# Cx
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cx[(i + 1) * n : (i + 2) * n, (i-j) * d: (i-j+1) * d ] = mat_temp
            mat_temp = A @ mat_temp
# Ay
    for i in range(N):
        Ay[i*r:(i+1)*r,:] = D @ matrix_power(A,i) 
# By
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            By[(i + 1) * r : (i + 2) * r, (i-j) * m: (i-j+1) * m ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Cy
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cy[(i + 1) * r : (i + 2) * r, (i-j) * d: (i-j+1) * d ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Ey
    for i in range(N):
        Ey[i * r : (i+1)*r, i * d : (i+1) * d] = E
# Cx_tilde
    Cx_tilde[ : , 0 : 1 ] = Ax @ x_init
    Cx_tilde[ : , 1 : ] = Cx
# Cy_tilde
    Cy_tilde[ r : , 0 : 1 ] = Ay @ x_init
    Cy_tilde[ r : , 1 : ] = Cy
# Ey_tilde
    Ey_tilde[0, 0] = 1
    Ey_tilde[1:, 1:] = Ey
# D_tilde
    for i in range(N):
        D_tilde[1 + i * r : 1 + (i+1) * r, i * d : (i+1) * d] = D
        
    return n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde 

Continuous-time system

$\begin{aligned}\left[\begin{array}{c}\dot{x}_{1}(t) \\ \dot{x}_{2}(t)\end{array}\right] &=\left[\begin{array}{cc}0 & 1 \\ -k_{2} / m & -k_{1} / m\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right]+\left[\begin{array}{c}0 \\ 1 / m\end{array}\right] u(t) \\ y(t) &=\left[\begin{array}{ll}1 & 0\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right] \end{aligned}$

In [7]:
m = 2 #[kg]
k1 = 3 # [N/m]
k2 = 2 # [N/m]

para = [m, k1, k2]

x_init = np.array([[-0.48356838],[0]])

In [8]:
A = np.array([[0,1],[-k2/m, -k1/m]])
B = np.array([[0],[1/m]])
delta_t = 0.1

In [9]:
Ak,Bk = disc_linear_system(A,B,delta_t)

In [10]:
Ak

array([[ 0.99137931,  0.0862069 ],
       [-0.0862069 ,  0.86206897]])

In [11]:
Bk

array([[0.00431034],
       [0.04310345]])

In [12]:
Ck = np.array([[1e-3, 0],[0, 0]])
D = np.array([[1, 0]])
E = np.array([[0,1e-3]])

N = 10
n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak,Bk,Ck,D,E, x_init, N)

In [13]:
Q = np.diag([10,1])
Qf = np.diag([15,1])
R = np.diag([1])

In [14]:
neg_K = dlqr(Ak,Bk,Q,R)

In [15]:
neg_K

matrix([[-1.64465128, -1.02359576]])

In [16]:
N_sim = 100


t0 = 0

xk = x_init
uk = 0
t = t0
h = delta_t

x_list = []
x_list += [xk.flatten().tolist()]
u_list = []
for i in range(N_sim):
    wk = np.random.randn(d,1)    
    uk = neg_K @ xk
    u_list +=  uk.flatten().tolist() 
    x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
    x_list += x_kp1.flatten().tolist()
    xk = x_kp1
    xk += Ck @ wk

In [17]:
u_list

[[0.7953013562120019],
 [0.704948328070747],
 [0.6191600756295695],
 [0.5415160674690147],
 [0.4653931285872529],
 [0.39731378353118313],
 [0.3355564262956343],
 [0.2756722421082905],
 [0.2236363411002963],
 [0.17857413799075172],
 [0.13862443193653934],
 [0.10258815962751466],
 [0.07138328454063719],
 [0.04555136293268386],
 [0.022138096959599257],
 [0.004602259370096395],
 [-0.0110584842994636],
 [-0.021979637064174024],
 [-0.035626613498100645],
 [-0.04140180382306205],
 [-0.046315880679267354],
 [-0.05041054463815106],
 [-0.051890938636427346],
 [-0.05550239623704692],
 [-0.05567524207647043],
 [-0.04981843843882362],
 [-0.04935084177334791],
 [-0.04798660020636261],
 [-0.045835418927814106],
 [-0.04291395894349295],
 [-0.03839221692454997],
 [-0.0346143156217672],
 [-0.03124674130269907],
 [-0.030199247109928416],
 [-0.02655782772486712],
 [-0.023805856263259612],
 [-0.020604293379789515],
 [-0.019728733716515876],
 [-0.017409861487296955],
 [-0.013743665560524958],
 [-0.009414089

In [18]:
x_list

[[-0.48356838, 0.0],
 [-0.4793779997914022, 0.08169497327379889],
 [-0.4679463099952694, 0.14707340188906431],
 [-0.45067197221496835, 0.198009315132406],
 [-0.4306996308793492, 0.2365830474523845],
 [-0.40513282079815255, 0.2640154358440708],
 [-0.37854610638107405, 0.28209218989565016],
 [-0.350842322909739, 0.2922683929882514],
 [-0.3201063010361907, 0.29541683028600707],
 [-0.2903993209982334, 0.29294890108301297],
 [-0.26193761675268945, 0.28606462050858444],
 [-0.2342313885507123, 0.27566947606663106],
 [-0.20703481713732363, 0.262467807483876],
 [-0.18127220105265282, 0.24719759418195902],
 [-0.1576586730418137, 0.2305888930240281],
 [-0.1347980224535936, 0.2130054193337578],
 [-0.11496664184905622, 0.19513485866121924],
 [-0.09611428569402897, 0.17719454743904917],
 [-0.0800820397199607, 0.15967735277122166],
 [-0.06262962957522801, 0.14226683512285782],
 [-0.04997822664558984, 0.12576241370942268],
 [-0.03832312282296016, 0.11017451388181018],
 [-0.027644673718352518, 0.095508

### System formulation
#### plant:

$$
\begin{aligned}
x_{k+1} &=A x_{k}+B u_{k} + C w_{k}\\
y_{k} &=D x_{k} + E w_{k}
\end{aligned}
$$
with $x_k \in \mathbb{R}^{n}, u_k \in \mathbb{R}^{m}, y_k \in \mathbb{R}^{r}, w_k \in \mathbb{R}^{d}$. Initial state $x_0$.


#### model:

$$
\begin{aligned}
\hat{x}_{k+1} &=A \hat{x}_{k}+B u_{k}\\
\hat{y}_{k} &=D \hat{x}_{k}
\end{aligned}
$$

Initial state $0$


### System behabior in N step
Define: $$
\boldsymbol{X}=\left[\begin{array}{llll}
x_{k}^{\top}, x_{k+1}^{\top} & \ldots & x_{k+N}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{U}=\left[\begin{array}{llll}
u_{k}^{\top} & u_{k+1}^{\top} & \ldots & u_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{Y}=\left[\begin{array}{llll}
y_{k}^{\top} & y_{k+1}^{\top} & \ldots & y_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{Y}_r=\left[\begin{array}{llll}
yr_{k}^{\top} & yr_{k+1}^{\top} & \ldots & yr_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

$$
\boldsymbol{W}=\left[\begin{array}{llll}
w_{k}^{\top} & w_{k+1}^{\top} & \ldots & w_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

$$
\tilde{\boldsymbol{W}}=\left[\begin{array}{llll}
1 & w_{k}^{\top} & \ldots & w_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

#### Predicted system
$$
\begin{array}{l}
\boldsymbol{X}=\mathcal{A}_{x} x_{k}+\mathcal{B}_{x} \boldsymbol{U} + \mathcal{C}_{x} \boldsymbol{W} \\
\boldsymbol{Y}=\mathcal{A}_{y} x_{k}+\mathcal{B}_{y} \boldsymbol{U} + \mathcal{C}_{y} \boldsymbol{W} + \mathcal{E}_{y} \boldsymbol{W}
\end{array}
$$
with
$$
\mathcal{A}_{x}=\left[\begin{array}{l}
\mathbb{I}_n \\ A^{1}\\  \vdots \\A^{N}
\end{array}\right]
$$

$$
\mathcal{B}_{x}=\left[\begin{array}{ccccc}
0 & 0 & \ldots & \ldots & 0\\
A^{0} B & 0 & \ldots & \ldots & 0 \\
A^{1} B & A^{0} B & \ddots & & \vdots \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & & \ddots & \ddots & 0 \\
A^{N-1} B & A^{N-2} B & \ldots & \ldots & A^{0} B
\end{array}\right]
$$

$$
\mathcal{C}_{x}=\left[\begin{array}{ccccc}
0 & 0 & \ldots & \ldots & 0\\
A^{0} C & 0 & \ldots & \ldots & 0 \\
A^{1} C & A^{0} C & \ddots & & \vdots \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & & \ddots & \ddots & 0 \\
A^{N-1} C & A^{N-2} C & \ldots & \ldots & A^{0} C
\end{array}\right]
$$

$$
\mathcal{A}_{y}=\left[\begin{array}{l}
D \\ DA^{1} \\ \vdots \\DA^{N-1}
\end{array}\right]
$$

$$
\mathcal{B}_{y}=\left[\begin{array}{cccccc}
0 & 0 & \ldots & \ldots & 0 & 0\\
DA^{0} B & 0 & \ldots & \ldots & 0 & 0\\
DA^{1} B & DA^{0} B & \ddots & & \vdots & 0\\
\vdots & \ddots & \ddots & \ddots & \vdots & 0\\
\vdots & & \ddots & \ddots & 0 & 0\\
DA^{N-2} B & DA^{N-3} B & \ldots & \ldots & DA^{0} B & 0
\end{array}\right]
$$

$$
\mathcal{C}_{y}=\left[\begin{array}{cccccc}
0 & 0 & \ldots & \ldots & 0 & 0\\
DA^{0} C & 0 & \ldots & \ldots & 0 & 0\\
DA^{1} C & DA^{0} C & \ddots & & \vdots & 0\\
\vdots & \ddots & \ddots & \ddots & \vdots & 0\\
\vdots & & \ddots & \ddots & 0 & 0\\
DA^{N-2} C & DA^{N-3} C & \ldots & \ldots & DA^{0} C & 0
\end{array}\right]
$$

$$
\mathcal{E}_{y} = \left[
  \begin{array}{ccc}
    E & & & & \\
    & \ddots & & & \\
    & & \ddots & & \\
    & & & \ddots & \\
    & & & & E
  \end{array} 
  \right]
$$


#### System with extended $\tilde{\boldsymbol{W}}$

$$
\begin{array}{l}
\boldsymbol{X}=\mathcal{B}_{x} \boldsymbol{U} + \tilde{\mathcal{C}}_{x} \tilde{\boldsymbol{W}} \\
\end{array}
$$

$$
\tilde{\mathcal{C}}_{x}=\left[\begin{array}{cccccc}
x_k & 0 & 0 & \ldots & \ldots & 0\\
A x_k & A^{0} C & 0 & \ldots & \ldots & 0 \\
A^2 x_k& A^{1} C & A^{0} C & \ddots & & \vdots \\
\vdots & \vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & \vdots & & \ddots & \ddots & 0 \\
A^N x_k & A^{N-1} C & A^{N-2} C & \ldots & \ldots & A^{0} C
\end{array}\right]
$$

#### puried-output-based (POB) affine control laws

$$
y - \hat{y} = \left(
\left[\begin{array}{c|cccc}
DA^0 x_0 & 0 & \dots & 0 & 0\\
DA^1 x_0 & DC & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
DA^{N-1} x_0 & DA^{N-2}C & \dots & DA^{0}C & 0\\
\end{array}\right]
+
\left[\begin{array}{c|cccc}
0 & E & 0 & \dots & 0\\
0 & 0 & E & \ddots & 0\\
\vdots &  & \ddots & \ddots & 0\\
0 & 0 & \dots & 0 & E\\
\end{array}\right]
\right)
\left[\begin{array}{c}
1\\
w_0\\
w_1\\
\vdots\\
w_{N-1}\\
\end{array}\right]
$$




$u_{k}=h_{k}+\sum_{\tau=0}^{k} H_{k \tau} v_{\tau}$, where $v_{k} = y_k - \hat{y}_k$

$$
\begin{array}{rll}
u_k &= h_0 + H_{00} (y_k - \hat{y}_k) &= h_0 + H_{00} (Dx_0 +Ew_0)\\
u_{k+1} &= h_1 + H_{10} (y_k - \hat{y}_k) + H_{11} (y_{k+1} - \hat{y}_{k+1}) &= h_1 + H_{10} (Dx_0 +Ew_0) + H_{11} (DAx_0 + DCw_0 + E w_1) \\
\vdots &  & \vdots \\
u_{k+N-1} &= h_{N-1}+\sum_{\tau=0}^{N-1} H_{(N-1) \tau} v_{\tau} &= h_{N-1} +H_{(N-1) 0}(Dx_0 +Ew_0) + \dots + H_{(N-1) (N-1)} (DA^{N-1} x_0 + DA^{N-2}Cw_0 + \dots + DA^{0}Cw_{N-2}) + Ew_{N-1}
\end{array}
$$

$\rightarrow$

$$\boldsymbol{U} = 
\left[\begin{array}{c|cccc}
h_0 & H_{00} & \dots & 0 & 0\\
h_1 & H_{10} & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
h_{N-1} & H_{(N-1)0} & \dots & H_{(N-1)(N-2)} & H_{(N-1)(N-1)}\\
\end{array}\right]
\left(
\left[\begin{array}{c|cccc}
0 & 0 & \dots & 0 & 0\\
DA^0 x_0 & 0 & \dots & 0 & 0\\
DA^1 x_0 & DC & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
DA^{N-1} x_0 & DA^{N-2}C & \dots & DA^{0}C & 0\\
\end{array}\right]
+
\left[\begin{array}{c|cccc}
1 & 0 & \dots & 0 & 0\\
0 & E & 0 & \dots & 0\\
0 & 0 & E & \ddots & 0\\
\vdots &  & \ddots & \ddots & 0\\
0 & 0 & \dots & 0 & E\\
\end{array}\right]
\right)
\left[\begin{array}{c}
1\\
w_0\\
w_1\\
\vdots\\
w_{N-1}\\
\end{array}\right]
$$

$\rightarrow$
$$\boldsymbol{U} = \mathcal{H}(\tilde{\mathcal{C}}_y + \tilde{\mathcal{E}}_y) \tilde{\boldsymbol{W}} = H\tilde{\boldsymbol{W}} $$, where $ \tilde{\mathcal{C}}_y  = \left[\begin{array}{ccccc}
0 & & & & \\
D & 0 & & & \\
& D & 0 & & \\
& & \ddots & \ddots & \\
& & & D & 0
\end{array}\right] \tilde{\mathcal{C}}_x  = \tilde{\mathcal{D}} \tilde{C}_x$


#### Matrix shape
$\boldsymbol{X} \in \mathbb{R}^{(N+1)n \times 1} $,
$\boldsymbol{U} \in \mathbb{R}^{Nm \times 1} $,
$\boldsymbol{Y} \in \mathbb{R}^{Nr \times 1} $,
$\boldsymbol{W} \in \mathbb{R}^{Nd \times 1} $
$\tilde{\boldsymbol{W}} \in \mathbb{R}^{(Nd+1) \times 1} $

$\mathcal{A}_{x} \in \mathbb{R}^{(N+1)n \times n}$,
$\mathcal{B}_{x} \in \mathbb{R}^{(N+1)n \times Nm}$,
$\mathcal{C}_{x} \in \mathbb{R}^{(N+1)n \times Nd}$,
$\mathcal{A}_{y} \in \mathbb{R}^{Nr \times n}$,
$\mathcal{B}_{y} \in \mathbb{R}^{Nr \times Nm}$,
$\mathcal{C}_{y} \in \mathbb{R}^{Nr \times Nd}$,
$\mathcal{E}_{y} \in \mathbb{R}^{Nr \times Nd}$

$\tilde{\mathcal{C}}_{x} \in \mathbb{R}^{(N+1)n \times (Nd+1)}$, $\tilde{\mathcal{C}}_{y} \in \mathbb{R}^{(Nr + 1) \times (Nd+1)}$, $\tilde{\mathcal{E}}_{y} \in \mathbb{R}^{(Nr + 1) \times (Nd+1)}$, $\tilde{\mathcal{D}} \in \mathbb{R}^{(Nr + 1) \times (N+1)n}$

$\mathcal{H} \in \mathbb{R}^{Nm \times (Nr + 1)}$


### Cost
#### Weighted stage cost
$\beta^{t} (x_{k+t}^{\top} Q x_{k+t} + u_{k+t}^{\top} R u_{k+t} )$
#### Cost
$J_{N}\left(x, \pi_{N}\right):=\sup _{\mathbb{P} \in \mathcal{P}_{\infty}} \mathbb{E}_{\mathbb{P}}\left\{\sum_{t=0}^{N-1} \beta^{t}\left[\boldsymbol{x}_{t}^{\top} Q \boldsymbol{x}_{t}+\boldsymbol{u}_{t}^{\top} R \boldsymbol{u}_{t}\right]+\beta^{N} \boldsymbol{x}_{N}^{\top} Q_{f} \boldsymbol{x}_{N}\right\}$
#### Reformulation with known first and second moments
$$\boldsymbol{U} = \mathcal{H}(\tilde{\mathcal{C}}_y + \tilde{\mathcal{E}}_y) \tilde{\boldsymbol{W}} = H\tilde{\boldsymbol{W}}$$ and $$\boldsymbol{X}=\mathcal{B}_{x} \boldsymbol{U} + \tilde{\mathcal{C}}_{x} \tilde{\boldsymbol{W}}$$

$\tilde{J}_{N}(x, H):=\operatorname{Tr}\left\{\left[(\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H)^{\top} J_{x} (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H) + H^{\top} J_u H \right]M_w\right\} + \mu_w^{\top} \left[ (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H)^{\top} J_{x} (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H) + H^{\top} J_u H \right]\mu_w$ with $J_x = \left[ \begin{array}{ccccc} \beta^{0}Q&&&&\\ &\beta^{1}Q&&&\\ &&\ddots&& \\ &&&\beta^{N-1}Q&\\ &&&&\beta^{N}Q_f \end{array}\right]$, $J_u = \left[ \begin{array}{cccc} \beta^{0}R&&&\\ &\beta^{1}R&&\\ &&\ddots& \\ &&&\beta^{N-1}R\end{array}\right]$, $M_{w}:=\left(1, \mu^{\top}, \ldots, \mu^{\top}\right)^{\top}\left(1, \mu^{\top}, \ldots, \mu^{\top}\right)+\operatorname{diag}\left(0, \mathbb{I}_{N} \otimes \mathbb{I}_{d}\right)$ and $\mu_w = \left[ \begin{array}{c} 1\\ \mu \\ \vdots \\ \mu \end{array} \right]$

### Chance constraint using wasserstein metric
Constraint on ith element of state: $[x_{k+n}]_i < v_{ubi}$


$\sup _{\mathbb{Q} \in \mathbb{B}_{\varepsilon}\left(\widehat{\mathbb{P}}_{N}\right)} \mathbb{E}^{\mathrm{Q}}[\ell(\xi)]$ with $l(w) = \max_{k \le N-1} \langle a_k, w\rangle + b_k$

equivalent to
$\begin{array}{lll}\inf _{\lambda, s_{i}, \gamma_{i k}} & \lambda \varepsilon+\frac{1}{N} \sum_{i=1}^{N} s_{i} \\ \text { s.t. } & b_{k}+\left\langle a_{k}, \widehat{\xi}_{i}\right\rangle+\left\langle\gamma_{i k}, d-C \widehat{\xi}_{i}\right\rangle \leq s_{i} & \forall i \leq N, \forall k \leq K \\ & \left\|C^{\top} \gamma_{i k}-a_{k}\right\|_{*} \leq \lambda & \forall i \leq N, \forall k \leq K \\ & \gamma_{i k} \geq 0 & \forall i \leq N, \forall k \leq K\end{array}$


#### Define $a_k$ and $b_k$
For example: Constraint on $[x_{k+1}]_1$

We have $x_{k+1} = Ax_0 + Bu_0 + Cw_0$ and $u_0 = h_0 + H_{00}(Dx_0 + Ew_0)$ $\rightarrow$ $[x_{k+1}]_1 = e_1^{\top} (Ax_0 + B(h_0 + H_{00}(Dx_0 + Ew_0)) + Cw_0) = \langle e_1^{\top} (BH_{00}E + C), w_0 \rangle  + e_1^{\top} (Ax_0 + B(h_0 + H_{00}Dx_0))$ 

Similiarly, $x_{k+N} = A^N x_0 + A^{N-1}Bu_0 + \dots + A^{0}Bu_{N-1} + A^{N-1}Cw_0 + \dots + A^{0}Cw_{N-1}$ and $u_{n}=h_{n}+\sum_{\tau=0}^{n} H_{n \tau} v_{\tau}$, where $v_{\tau} = y_{\tau} - \hat{y}_{\tau} = DA^{\tau}x_0 + DA^{\tau-1}Cw_0 + \dots + DA^{0}C w_{\tau-1} + E w_{\tau}$

$\rightarrow$ $[x_{k+N}]_i = e^{\top}_i (A^Nx_0 + \sum_{n=0}^{N-1}A^{n}B \underbrace{(h_{N-1-n} + \sum_{\tau = 0}^{N - 1 - n} ( H_{n\tau} \underbrace{( DA^{\tau}x_0 + \sum_{\kappa = 0}^{\tau-1} DA^{\kappa}Cw_{\tau-1 - \kappa} + Ew_{\tau})}_{v_\tau}))}_{u_{N- 1 - n}} + \sum_{n=0}^{N-1} A^{n}Cw_{N-1-n}) $

$ [x_{k+N}]_i = \langle e_i^{\top}(BH_{(N-1)(N-1)}E + C), w_{N-1} \rangle + e^{\top}_i (A^Nx_0 + \sum_{n=0}^{N-1}A^{n}B(h_{N-1-n} + \sum_{\tau = 0}^{N - 1 - n} ( H_{n\tau}( DA^{\tau}x_0 + \sum_{\kappa = 0}^{\tau-1} DA^{\kappa}Cw_{\tau-1 - \kappa}))) + \sum_{n=1}^{N-1} A^{n}Cw_{N-1-n})$

In [19]:
# def stack_system(A, B, C, D, E, x_init, N):
#     '''
#     Stack system matrix for N prediction horizon
    
#     x_next = A x_k + B u_k + C w_k
#     y_k = D x_k + E w_k
    
#     '''

#     n = np.shape(A)[1]    #  Dimension of state
#     m = np.shape(B)[1]    #  Dimension of input
#     d = np.shape(C)[1]    #  Dimension of disturbance
#     r = np.shape(D)[0]    #  Dimension of output
    
# #     print(Nx,Nu,Nw,Ny)
#     Nx = (N+1) * n
#     Nu = N * m
#     Ny = N * r
#     Nw = N * d
    
#     Ax = np.zeros([Nx,n])
#     Bx = np.zeros([Nx,Nu])
#     Cx = np.zeros([Nx,Nw])
#     Ay = np.zeros([Ny,n])
#     By = np.zeros([Ny,Nu])
#     Cy = np.zeros([Ny,Nw])
#     Ey = np.zeros([Ny,Nw])
#     Cx_tilde = np.zeros([Nx, Nw + 1])
#     Cy_tilde = np.zeros([Ny + 1, Nw + 1])
#     Ey_tilde = np.zeros([Ny + 1, Nw + 1])
#     D_tilde = np.zeros([Ny + 1, Nx])
# #     H

# # Ax
#     for i in range(N+1):
#         Ax[i*n:(i+1)*n,:] = matrix_power(A,i)
# # Bx
#     for i in range(N):
#         mat_temp = B
#         for j in range (i+1):
#             Bx[(i + 1) * n : (i + 2) * n, (i-j) * m: (i-j+1) * m ] = mat_temp
#             mat_temp = A @ mat_temp
# # Cx
#     for i in range(N):
#         mat_temp = C
#         for j in range (i+1):
#             Cx[(i + 1) * n : (i + 2) * n, (i-j) * d: (i-j+1) * d ] = mat_temp
#             mat_temp = A @ mat_temp
# # Ay
#     for i in range(N):
#         Ay[i*r:(i+1)*r,:] = D @ matrix_power(A,i) 
# # By
#     for i in range(N):
#         mat_temp = B
#         for j in range (i+1):
#             By[(i + 1) * r : (i + 2) * r, (i-j) * m: (i-j+1) * m ] = D @ mat_temp
#             mat_temp = A @ mat_temp
# # Cy
#     for i in range(N):
#         mat_temp = C
#         for j in range (i+1):
#             Cy[(i + 1) * r : (i + 2) * r, (i-j) * d: (i-j+1) * d ] = D @ mat_temp
#             mat_temp = A @ mat_temp
# # Ey
#     for i in range(N):
#         Ey[i * r : (i+1)*r, i * d : (i+1) * d] = E
# # Cx_tilde
#     Cx_tilde[ : , 0 : 1 ] = Ax @ x_init
#     Cx_tilde[ : , 1 : ] = Cx
# # Cy_tilde
#     Cy_tilde[ r : , 0 : 1 ] = Ay @ x_init
#     Cy_tilde[ r : , 1 : ] = Cy
# # Ey_tilde
#     Ey_tilde[0, 0] = 1
#     Ey_tilde[1:, 1:] = Ey
# # D_tilde
#     for i in range(N):
#         D_tilde[1 + i * r : 1 + (i+1) * r, i * d : (i+1) * d] = D
        
#     return n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde 

In [20]:
# Ck = np.array([[1e-3, 0],[0, 0]])
# D = np.array([[1, 0]])
# E = np.array([[0,1e-3]])

# N = 5
# n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak,Bk,Ck,D,E, x_init, N)

In [21]:
Ax

array([[ 1.        ,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.99137931,  0.0862069 ],
       [-0.0862069 ,  0.86206897],
       [ 0.97540131,  0.15978002],
       [-0.15978002,  0.73573127],
       [ 0.95321854,  0.22182772],
       [-0.22182772,  0.62047696],
       [ 0.92587806,  0.2734048 ],
       [-0.2734048 ,  0.51577085],
       [ 0.89432697,  0.31551087],
       [-0.31551087,  0.42106066],
       [ 0.85941804,  0.34908928],
       [-0.34908928,  0.33578412],
       [ 0.82191536,  0.3750268 ],
       [-0.3750268 ,  0.25937516],
       [ 0.78249999,  0.39415374],
       [-0.39415374,  0.19126938],
       [ 0.74177553,  0.4072446 ],
       [-0.4072446 ,  0.13090863],
       [ 0.70027362,  0.4150191 ],
       [-0.4150191 ,  0.07774497]])

In [22]:
Bx

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.04310345, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.007989  , 0.00431034, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.03678656, 0.04310345, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01109139, 0.007989  , 0.00431034, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.03102385, 0.03678656, 0.0431034

In [23]:
Cx

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.0

In [24]:
Ay

array([[1.        , 0.        ],
       [0.99137931, 0.0862069 ],
       [0.97540131, 0.15978002],
       [0.95321854, 0.22182772],
       [0.92587806, 0.2734048 ],
       [0.89432697, 0.31551087],
       [0.85941804, 0.34908928],
       [0.82191536, 0.3750268 ],
       [0.78249999, 0.39415374],
       [0.74177553, 0.4072446 ]])

In [25]:
By

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.007989  , 0.00431034, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01109139, 0.007989  , 0.00431034, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01367024, 0.01109139, 0.007989  , 0.00431034, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01577554, 0.01367024, 0.01109139, 0.007989  , 0.00431034,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01745446, 0.01577554, 0.01367024, 0.01109139, 0.007989  ,
        0.00431034, 0.        , 0.        , 0.        , 0.        ],
       [0.01875134, 0.01745446, 0.0157755

In [26]:
Cy

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.001     , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00099138, 0.        , 0.001     , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.0009754 , 0.        , 0.00099138, 0.        , 0.001     ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0

In [27]:
Ey

array([[0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.

In [28]:
Cx_tilde

array([[-4.83568380e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.79399687e-01,  1.00000000e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e

In [29]:
Cy_tilde

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.48356838,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.47939969,  0.001     ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.47167323,  0.00099138,  0.        ,  0.001     

In [30]:
Ey_tilde

array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   

In [31]:
D_tilde

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [32]:
def define_loss_func(m, d, r, Nu, Nw, Bx, Cx_tilde, mu, beta):
    # Define decision variables for POB affine constrol law
    H_cal_dec = cp.Variable((Nu , 1))
#     print(H_cal_dec)
    # \mathcal{H} matrix
    for i in range(N):
        H_col = cp.Variable((Nu - (i * m) , r))
        if i > 0:
            H_col = cp.vstack([np.zeros([i * m, r]), H_col])
#         print(H_col)
        H_cal_dec = cp.hstack([H_cal_dec,H_col])
#     print(H_cal_dec)
#     print(np.shape(H_cal_dec))
    # Define intermediate decision variables for objective function
    H = cp.Variable((Nu , Nw + 1))
    
    # Define loss function
#     beta = 0.95
    Jx = np.zeros([(N+1)* n, (N+1)* n])
    for i in range(N):
        Jx[i * n : (i+1) * n, i * n : (i+1) * n] = beta**i * Q
    Jx[ N * n: , N * n:] = beta ** N * Qf

    Ju = np.zeros([N * m, N * m])
    for i in range(N):
        Ju[i * m : (i+1) * m, i * m : (i+1) * m] = beta**i * R

    # This is only for var = 1 and mean = 0. Should be modified.
    mu_w = np.vstack([1]+ [mu] * N)
    M_w = mu_w @ mu_w.T + np.diag([0] + [1] * N * d)
    # Intermediate decision variables. Since CVXPY does not support quadratic obj of decision variable matrix.
    H_new_matrix = []
    for i in range(Nw+1):
        H_new_matrix += [cp.Variable([Nu,1])]
    H_new = cp.hstack(H_new_matrix)
#     print(H_new.shape)
    # Reformulate the quadratic term
    eigval,eigvec = np.linalg.eig((Ju + Bx.T @ Jx @ Bx))
    eigval_mat =  np.diag(eigval)
#     print(Ju + Bx.T @ Jx @ Bx - eigvec @ eigval_mat @ np.linalg.inv(eigvec))
    # Loss function
    loss_func = 0
    
    N_eig = np.shape(eigval)[0]
    I = np.diag([1]*N_eig)
    for i in range(N_eig):
        # Reformulate Tr(H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ M_w
        loss_func += eigval[i] * cp.quad_form(H_new_matrix[i], I) # When M_w is identity matrix. Otherwise reformulate system matrix or this line
#     loss_func += cp.trace(2 * Cx_tilde.T @ Jx @ Bx @ H)
    # Reformulate mu_w.T @ (H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ mu_w
#     loss_func += eigval[0] * cp.quad_form(H_new_matrix[0], I) +  2 * mu_w.T @  Cx_tilde.T @ Jx @ Bx @ H @ mu_w

    
    return Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func

In [33]:
def gene_disturbance(N, d, N_sample):
    # Generate data 
#     N_sample = 6

    w_sample = []
    for i in range(N_sample):
        w_temp = np.random.randn(N*d)
        w_sample += [w_temp]
    W_sample_matrix = np.array(w_sample).T

    W_sample_matrix_ext = np.vstack( [np.ones([1, N_sample]),W_sample_matrix])
    return W_sample_matrix, W_sample_matrix_ext

In [34]:
def define_constraint(W_sample_matrix, W_sample_matrix_ext, eigvec, H_cal_dec, H, H_new, n, Bx, Cx_tilde, Cy_tilde, Ey_tilde, i_th_state = 1, i_state_ub = 0.05, epsilon = 1, N_sample = 6):
    X_constraint = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde) @ W_sample_matrix_ext
    constraint = []
#     constraint += [H_new == np.linalg.inv(eigvec) @ H ]
    constraint += [H_new == eigvec.T @ H ]
    constraint += [H == H_cal_dec @ (Cy_tilde + Ey_tilde) ]
    
#     i_th_state = 1 # 0 for first element, 1 for second element
#     i_state_ub = 0.05

    d_supp = 3 * np.ones([N*d, 1])
    C_supp = np.diag([1]*N*d)

#     epsilon = 5 # select 20 best

    lambda_var = cp.Variable()

    gamma_matrix = []
    for i in range(N_sample):
        for j in range(N):
            gamma_var = cp.Variable([N*d,1])
            gamma_matrix += [gamma_var]

    si_var = cp.Variable([N_sample,1])
    for i in range(N_sample):
        for j in range(N):
            constraint_temp = X_constraint[n * (j+1) + i_th_state,i] + gamma_matrix[i * N + j].T @ (d_supp - C_supp @ W_sample_matrix[:, i])
            constraint += [constraint_temp <= si_var[i,0]]


    ak_matrix = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde)[:,1:]
    for i in range(N_sample):
        for j in range(N):
            constraint_temp = C_supp.T @ gamma_matrix[i * N + j] - ak_matrix[n * (j+1) + i_th_state:n * (j+1)+i_th_state + 1,:].T
            constraint += [cp.norm_inf(constraint_temp) <= lambda_var]

    for i in range(N_sample):
        for j in range(N):
            constraint += [gamma_matrix[i * N + j] >= 0]
    constraint += [lambda_var * epsilon + 1/N_sample * cp.sum(si_var) <= i_state_ub]
    return lambda_var, gamma_matrix, si_var, constraint

In [35]:
def define_obj(n, m, d, r, Nu, Nw, Bx, Cx_tilde, Cy_tilde, Ey_tilde, mu, beta = 0.95, N_sample = 6, i_th_state = 1, i_state_ub = 0.05, epsilon = 1):
    Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func = define_loss_func(m, d, r, Nu, Nw, Bx, Cx_tilde, mu, beta)
    W_sample_matrix, W_sample_matrix_ext = gene_disturbance(N, d, N_sample)
    lambda_var, gamma_matrix, si_var, constraint = define_constraint(W_sample_matrix, W_sample_matrix_ext, eigvec, H_cal_dec, H, H_new, n, Bx, Cx_tilde, Cy_tilde, Ey_tilde, i_th_state, i_state_ub, epsilon, N_sample)
    return Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func, W_sample_matrix, W_sample_matrix_ext, lambda_var, gamma_matrix, si_var, constraint

In [36]:
mu =np.zeros([d,1])
mu_w = np.vstack([1]+ [mu] * N)
M_w = mu_w @ mu_w.T + np.diag([0] + [1] * N * d)
beta = 0.95
N_sample = 5
i_th_state = 1 
i_state_ub = 0.5
epsilon = 0.05

In [37]:
# Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func, W_sample_matrix, W_sample_matrix_ext, lambda_var, gamma_matrix, si_var, constraint= define_obj(n, m, d, r, Nu, Nw, Bx, Cx_tilde, Cy_tilde, Ey_tilde, mu, beta = beta, N_sample = N_sample, i_th_state = i_th_state, i_state_ub = i_state_ub, epsilon = epsilon)

# obj = cp.Minimize(loss_func)
# prob = cp.Problem(obj,constraint)
# prob.solve()

# lambda_var.value

# si_var.value

# np.sum(si_var.value)/N_sample + lambda_var.value * epsilon

# H_new_matrix[0].value

# H_cal_dec.value

# np.trace(Cx_tilde.T @ Jx @ Cx_tilde) + mu_w.T @ Cx_tilde.T @ Jx @ Cx_tilde @mu_w

# np.trace(((Cx_tilde+Bx @ H.value).T @ Jx @ (Cx_tilde+Bx @ H.value) + H.value.T @ Ju @ H.value)@ M_w) + mu_w.T @ ((Cx_tilde+Bx @ H.value).T @ Jx @ (Cx_tilde+Bx @ H.value) + H.value.T @ Ju @ H.value) @ mu_w

# H.value @ W_sample_matrix_ext

# Ax @ x_init +  Bx @ H.value @ W_sample_matrix_ext[:,0:1]  + Cx @ W_sample_matrix[:,0:1]

# Ck

# H_cal_dec.value[0,0]

# H_cal_dec.value[0,1]

# 10 % 10 == 0

# x_init

# prob.value

In [39]:
N_sim = 100


t0 = 0

xk = x_init
uk = 0
t = t0
h = delta_t



x_list = []
x_list += [xk.flatten().tolist()]
u_list = []
for i in range(N_sim):
    if i % N == 0:
        n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak, Bk, Ck, D, E, xk, N)
        Jx, Ju, eigval,eigvec, H_cal_dec, H, H_new_matrix, H_new, loss_func, W_sample_matrix, W_sample_matrix_ext, lambda_var, gamma_matrix, si_var, constraint = define_obj(n, m, d, r, Nu, Nw, Bx, Cx_tilde, Cy_tilde, Ey_tilde, mu, beta = beta, N_sample = N_sample, i_th_state = i_th_state, i_state_ub = i_state_ub, epsilon = epsilon)
        obj = cp.Minimize(loss_func)
        prob = cp.Problem(obj,constraint)
#         print(W_sample_matrix)
        print( prob.solve())
#         print(Ax @ x_init +  Bx @ H.value @ W_sample_matrix_ext[:,0:1]  + Cx @ W_sample_matrix[:,0:1])
#         print("status:", prob.status)
#         print("Controller", H_cal_dec.value[0,0], H_cal_dec.value[0,1])
#         print("lambda",lambda_var.value)
#         print("si",si_var.value)
#         print("state_constraint", np.sum(si_var.value)/N_sample + lambda_var.value * epsilon)
        
    wk = np.random.randn(d,1)
    uk = H_cal_dec.value[0,0] + H_cal_dec.value[0,1] * (D @ xk + E @ wk)
    u_list +=  uk.flatten().tolist() 
    print(xk,uk)
    x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
    x_list += x_kp1.flatten().tolist()
    xk = x_kp1
    xk += Ck @ wk





2.2142141838192042e-07
[[-0.48356838]
 [ 0.        ]] [[-5.29351135e-10]]
[[-0.48166606]
 [ 0.04483006]] [[5.60047879e-10]]
[[-0.47581583]
 [ 0.08303652]] [[1.60431075e-09]]
[[-0.46688783]
 [ 0.11520594]] [[3.48794534e-09]]
[[-0.45474715]
 [ 0.14192122]] [[5.46198074e-09]]
[[-0.44013274]
 [ 0.16366891]] [[8.14642695e-09]]
[[-0.4236568 ]
 [ 0.18093409]] [[1.15362997e-08]]
[[-0.40447378]
 [ 0.19418884]] [[1.53500183e-08]]
[[-0.38433905]
 [ 0.20375896]] [[1.95007838e-08]]
[[-0.3638128 ]
 [ 0.21008612]] [[2.29577617e-08]]
1.0280774860701517e-07
[[-0.34372854]
 [ 0.21360042]] [[4.1766887e-11]]
[[-0.32200131]
 [ 0.21474736]] [[5.46665169e-08]]
[[-0.30065955]
 [ 0.2137151 ]] [[1.21822547e-07]]
[[-0.27819656]
 [ 0.21085276]] [[1.86065751e-07]]
[[-0.25673112]
 [ 0.20631961]] [[2.48013167e-07]]
[[-0.23555114]
 [ 0.2004484 ]] [[3.10834574e-07]]
[[-0.21607752]
 [ 0.19345804]] [[3.66845287e-07]]
[[-0.19850272]
 [ 0.18566767]] [[4.23137467e-07]]
[[-0.18089059]
 [ 0.17736837]] [[4.69707039e-07]]
[[-0

TypeError: 'NoneType' object is not subscriptable